# CheXpert Dataset Preprocessing Guide

This notebook provides a comprehensive guide for downloading and preprocessing the CheXpert large chest X-ray dataset for for ART-ASyn.

## Dataset Information
- **Sources**:
    - [Stanford AIMI - Full Resolution CheXpert Train & Validation Dataset](https://stanfordaimi.azurewebsites.net/datasets/8cbd9ed4-2eb9-4565-affc-111cf4f7ebe2)  OR  [Kaggle - Small CheXpert Train & Validation Dataset](https://www.kaggle.com/datasets/ashery/chexpert)
    - [Stanford AIMI - CheXpert Test Dataset](https://stanfordaimi.azurewebsites.net/datasets/23c56a0d-15de-405b-87c8-99c30138950c)
- **Type**: Large-scale chest X-ray dataset for multi-label classification
- **Format**: JPEG images with CSV annotations

## Dataset Structure
The CheXpert dataset is organized into three main splits:
- **Training Set**: `train.csv` with 191,027 frontal images
- **Validation Set**: `valid.csv` with 202 frontal images  
- **Test Set**: `test.csv` with 518 frontal images

### 13 Medical Conditions
The dataset includes labels for these chest pathologies:
1. **Enlarged Cardiomediastinum** - Heart and mediastinum enlargement
2. **Cardiomegaly** - Abnormal enlargement of the heart
3. **Lung Opacity** - Opacities or shadows in lung fields
4. **Lung Lesion** - Lung lesions or masses
5. **Edema** - Pulmonary edema
6. **Consolidation** - Lung consolidation
7. **Pneumonia** - Pneumonia detection
8. **Atelectasis** - Lung collapse
9. **Pneumothorax** - Collapsed lung
10. **Pleural Effusion** - Fluid in pleural space
11. **Pleural Other** - Other pleural abnormalities
12. **Fracture** - Bone fractures
13. **Support Devices** - Medical devices present

### Label Encoding System
- **1.0**: Positive (Abnormal finding present)
- **0.0**: Negative (Abnormal finding absent)
- **-1.0**: Uncertain (Uncertain/equivocal findings)
- **NaN**: Missing (Finding not mentioned)

This preprocessing script organizes the data into a binary classification format (healthy vs diseased) suitable for machine learning models.

---

In [1]:
# Required Libraries
import os
import shutil
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image

print("✅ All required libraries imported successfully")

def create_dir(dir_name):
    """Create directory if it doesn't exist"""
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

✅ All required libraries imported successfully


## Setup and Configuration

### Step 1: Download the Dataset

Before running this notebook, you need to download the CheXpert dataset. You have **two options**:

#### Option 1. Stanford AIMI (Complete Train & Validation Dataset)
1. **Visit**: https://stanfordaimi.azurewebsites.net/datasets/8cbd9ed4-2eb9-4565-affc-111cf4f7ebe2
2. **Download Size**: 471.17 GB (Complete dataset)
3. **Registration Required**: Must register for access
4. **Download Command** (after registration):
   ```bash
   azcopy copy <download_url> <download-path>/data --recursive=true
   ```

#### Option 2: Kaggle (Recommended)
1. **Visit**: https://www.kaggle.com/datasets/ashery/chexpert
2. **Download Size**: 10.7GB
3. **Direct Download**: Requires Kaggle account, download directly from the page
4. **Structure**: Complete CheXpert-v1.0-small dataset ready for use

#### Must download: Stanford AIMI - Test Dataset
1. **Visit**: https://stanfordaimi.azurewebsites.net/datasets/23c56a0d-15de-405b-87c8-99c30138950c
2. **Download Size**: 5.73GB
3. **Registration Required**: Must register for access
4. **Download Command** (after registration):
   ```bash
   azcopy copy <download_url> <download-path>/data --recursive=true
   ```
5. **File Structure After Download**:
   ```
   chexlocalize/
   └── CheXpert/
       ├── test/
       │   └── patient*/study*/images
       └── test_labels.csv  # Rename to test.csv
   ```
6. **Important**:
    - Rename `test_labels.csv` to `test.csv` and place it inside the Kaggle dataset's `CheXpert-v1.0-small` folder
    - Copy `test` folder and place it inside the Kaggle dataset's `CheXpert-v1.0-small` folder

### Step 2: Extract and Organize Dataset

Extract the dataset to create the following folder structure:
```
CheXpert-v1.0-small/     # Main dataset folder
├── train.csv           # Training labels
├── valid.csv           # Validation labels
├── test.csv            # Test labels (from Stanford AIMI)
├── train/
│   └── patient*/study*/images/*.jpg
├── valid/
│   └── patient*/study*/images/*.jpg
└── test/
    └── patient*/study*/images/*.jpg
```

**Note**: The main notebook directory should be at the same level as the `CheXpert-v1.0-small` folder:
```
data/
├── CheXpert/
│   └── Data Preprocessing.ipynb
└── CheXpert-v1.0-small/  # Dataset folder
```

### Step 3: Configure Data Paths

Set the root directory where your CheXpert dataset is located:

In [2]:
# Configure dataset root directory
# Change this path to match your local dataset location
root = "../CheXpert-v1.0-small"

# Define CSV file paths
train_csv = f"{root}/train.csv"
val_csv = f"{root}/valid.csv"
test_csv = f"{root}/test.csv"

# Verify the dataset directory exists
if Path(root).exists():
    print(f"✅ Dataset directory found: {root}")
    print(f"📁 Directory contents: {list(Path(root).iterdir())}")
    
    # Check if required CSV files exist
    csv_files = [train_csv, val_csv, test_csv]
    for csv_file in csv_files:
        if Path(csv_file).exists():
            print(f"✅ Found CSV: {Path(csv_file).name}")
        else:
            print(f"❌ Missing CSV: {Path(csv_file).name}")
else:
    print(f"❌ Dataset directory not found: {root}")
    print("Please ensure you have downloaded and extracted the dataset correctly.")
    print("Expected structure: ../CheXpert-v1.0-small/{train.csv,valid.csv,test.csv,train/,valid/,test/}")


✅ Dataset directory found: ../CheXpert-v1.0-small
📁 Directory contents: [WindowsPath('../CheXpert-v1.0-small/test'), WindowsPath('../CheXpert-v1.0-small/test.csv'), WindowsPath('../CheXpert-v1.0-small/train'), WindowsPath('../CheXpert-v1.0-small/train.csv'), WindowsPath('../CheXpert-v1.0-small/valid'), WindowsPath('../CheXpert-v1.0-small/valid.csv')]
✅ Found CSV: train.csv
✅ Found CSV: valid.csv
✅ Found CSV: test.csv


## Data Preprocessing

### Overview

The preprocessing pipeline will:
1. **Load CSV annotations** for train, validation, and test sets
2. **Filter frontal images only** (exclude lateral views)
3. **Classify images** as healthy or diseased based on pathology labels
4. **Handle uncertainty labels** (-1.0 values) appropriately
5. **Organize images** by class and dataset split
6. **Create structured directories** for ML workflows

### Classification Logic

Images are classified as:
- **Healthy**: No positive pathology findings (all labels are 0.0, -1.0, or NaN)
- **Diseased**: At least one positive pathology finding (label = 1.0)

**Uncertainty Handling**: Images with only -1.0 (uncertain) and 0.0 (negative) labels are excluded to ensure clean binary classification.

### Dataset Structure After Preprocessing

```
📂 CheXpert/
├── train/
│   ├── healthy/     # Images with no pathology findings
│   └── diseased/   # Images with at least one pathology
├── val/
│   ├── healthy/     # Validation healthy images
│   └── diseased/   # Validation diseased images
└── test/
    ├── healthy/     # Test healthy images (resized to 25%)
    └── diseased/   # Test diseased images (resized to 25%)
```

### File Naming Convention

CheXpert images follow the pattern: `{patient_id}/{study_id}/{image_name}.jpg`
- `patient_id`: Unique patient identifier
- `study_id`: Study session identifier  
- `image_name`: Specific image file name

**Example**: `patient00001/study01/view1_frontal.jpg`

Images are renamed during preprocessing to: `{patient_id}_{study_id}.{extension}`

## Training Data Preprocessing

The following code processes the training dataset:

In [3]:
# Load and analyze training data
print("🔄 Loading training data...")
print("=" * 50)

train_df = pd.read_csv(train_csv)

# Only select frontal images
frontal = train_df["Path"].str.endswith("_frontal.jpg")
train_df = train_df[frontal]

print(f"📊 Total frontal training images: {len(train_df):,}")

train_paths = train_df["Path"]

# Analyze pathology labels
print("\n🔍 Analyzing pathology labels...")
data = train_df.iloc[:, 6:].fillna(0)  # All columns of findings

# Classification logic
unsure_findings = (data.max(axis=1) == 0) & ((data.min(axis=1) == -1))  # Only uncertain findings
unhealthy_idxs = ~unsure_findings & data.max(axis=1) == 1  # Has positive findings
healthy_idxs = ~unsure_findings & ~unhealthy_idxs  # No positive findings

print(f"   Healthy images (no pathology): {sum(healthy_idxs):,}")
print(f"   Diseased images (has pathology): {sum(unhealthy_idxs):,}")
print(f"   Uncertain images (excluded): {sum(unsure_findings):,}")

print("\n✅ Training data analysis completed!")


🔄 Loading training data...
📊 Total frontal training images: 191,027

🔍 Analyzing pathology labels...
   Healthy images (no pathology): 12,234
   Diseased images (has pathology): 173,708
   Uncertain images (excluded): 5,085

✅ Training data analysis completed!


In [4]:
# # Process training data - Diseased images (Optional)
# print("\n🔄 Processing training data - Diseased images...")
# print("=" * 55)

# target_dir = "train/diseased"
# create_dir(target_dir)

# diseased_count = 0
# for path in tqdm(train_paths[unhealthy_idxs], desc="Processing diseased training images"):
#     source = f"../{path}"
#     _, _, patient_id, study_id, img_name = path.split("/")
#     suffix = img_name.split(".")[-1]
#     destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
#     if not os.path.exists(destination):
#         shutil.copy(source, destination)
#         diseased_count += 1

# print(f"✅ Processed {diseased_count:,} diseased training images")

In [5]:
# Process training data - Healthy images
print("\n🔄 Processing training data - Healthy images...")
print("=" * 54)

target_dir = "train/healthy"
create_dir(target_dir)

healthy_count = 0
for path in tqdm(train_paths[healthy_idxs], desc="Processing healthy training images"):
    source = f"../{path}"
    _, _, patient_id, study_id, img_name = path.split("/")
    suffix = img_name.split(".")[-1]
    destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
    if not os.path.exists(destination):
        shutil.copy(source, destination)
        healthy_count += 1

print(f"✅ Processed {healthy_count:,} healthy training images")


🔄 Processing training data - Healthy images...


Processing healthy training images: 100%|█████████████████████████████████████████████████| 12234/12234 [00:22<00:00, 542.99it/s]

✅ Processed 11,924 healthy training images


## Validation Data Preprocessing

The following code processes the validation dataset using the same methodology:

In [6]:
# Load and analyze validation data
print("🔄 Loading validation data...")
print("=" * 50)

val_df = pd.read_csv(val_csv)

# Only select frontal images
frontal = val_df["Path"].str.endswith("_frontal.jpg")
val_df = val_df[frontal]

print(f"📊 Total frontal validation images: {len(val_df):,}")

val_paths = val_df["Path"]

# Analyze pathology labels
print("\n🔍 Analyzing validation pathology labels...")
data = val_df.iloc[:, 6:].fillna(0)  # All columns of findings

# Classification logic (same as training)
unsure_findings = (data.max(axis=1) == 0) & ((data.min(axis=1) == -1))
unhealthy_idxs = ~unsure_findings & data.max(axis=1) == 1
healthy_idxs = ~unsure_findings & ~unhealthy_idxs

print(f"   Healthy images (no pathology): {sum(healthy_idxs):,}")
print(f"   Diseased images (has pathology): {sum(unhealthy_idxs):,}")
print(f"   Uncertain images (excluded): {sum(unsure_findings):,}")

print("\n✅ Validation data analysis completed!")

🔄 Loading validation data...
📊 Total frontal validation images: 202

🔍 Analyzing validation pathology labels...
   Healthy images (no pathology): 31
   Diseased images (has pathology): 171
   Uncertain images (excluded): 0

✅ Validation data analysis completed!


In [7]:
# Process validation data - Diseased images
print("\n🔄 Processing validation data - Diseased images...")
print("=" * 57)

target_dir = "val/diseased"
create_dir(target_dir)

diseased_count = 0
for path in tqdm(val_paths[unhealthy_idxs], desc="Processing diseased validation images"):
    source = f"../{path}"
    _, _, patient_id, study_id, img_name = path.split("/")
    suffix = img_name.split(".")[-1]
    destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
    if not os.path.exists(destination):
        shutil.copy(source, destination)
        diseased_count += 1

print(f"✅ Processed {diseased_count:,} diseased validation images")


🔄 Processing validation data - Diseased images...


Processing diseased validation images: 100%|██████████████████████████████████████████████████| 171/171 [00:00<00:00, 693.90it/s]

✅ Processed 170 diseased validation images


In [8]:
# Process validation data - Healthy images
print("\n🔄 Processing validation data - Healthy images...")
print("=" * 56)

target_dir = "val/healthy"
create_dir(target_dir)

healthy_count = 0
for path in tqdm(val_paths[healthy_idxs], desc="Processing healthy validation images"):
    source = f"../{path}"
    _, _, patient_id, study_id, img_name = path.split("/")
    suffix = img_name.split(".")[-1]
    destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
    if not os.path.exists(destination):
        shutil.copy(source, destination)
        healthy_count += 1

print(f"✅ Processed {healthy_count:,} healthy validation images")


🔄 Processing validation data - Healthy images...


Processing healthy validation images: 100%|█████████████████████████████████████████████████████| 31/31 [00:00<00:00, 731.80it/s]

✅ Processed 30 healthy validation images


## Testing Data Preprocessing

The following code processes the testing dataset. **Note**: Test images are resized to 25% of original size to reduce computational requirements.

In [9]:
# Define image resizing function
def resize(image, ratio=0.25):
    """Resize image to specified ratio"""
    w, h = image.size
    width = int(w * ratio)
    height = int(h * ratio)
    return image.resize((width, height))

In [10]:
# Load and analyze test data
print("🔄 Loading test data...")
print("=" * 50)

test_df = pd.read_csv(test_csv)

# Only select frontal images
frontal = test_df["Path"].str.endswith("_frontal.jpg")
test_df = test_df[frontal]

print(f"📊 Total frontal test images: {len(test_df):,}")

test_paths = test_df["Path"]

# Analyze pathology labels
print("\n🔍 Analyzing test pathology labels...")
data = test_df.iloc[:, 2:-1].fillna(0)  # All columns of findings

# Classification logic (same as training/validation)
unsure_findings = (data.max(axis=1) == 0) & ((data.min(axis=1) == -1))
unhealthy_idxs = ~unsure_findings & data.max(axis=1) == 1
healthy_idxs = ~unsure_findings & ~unhealthy_idxs

print(f"   Healthy images (no pathology): {sum(healthy_idxs):,}")
print(f"   Diseased images (has pathology): {sum(unhealthy_idxs):,}")
print(f"   Uncertain images (excluded): {sum(unsure_findings):,}")

print("\n✅ Test data analysis completed!")


🔄 Loading test data...
📊 Total frontal test images: 518

🔍 Analyzing test pathology labels...
   Healthy images (no pathology): 157
   Diseased images (has pathology): 361
   Uncertain images (excluded): 0

✅ Test data analysis completed!


In [11]:
# Process test data - Diseased images (with resizing)
print("\n🔄 Processing test data - Diseased images (resizing to 25%)...")
print("=" * 67)

target_dir = "test/diseased"
create_dir(target_dir)

diseased_count = 0
for path in tqdm(test_paths[unhealthy_idxs], desc="Processing diseased test images"):
    source = f"../CheXpert-v1.0-small/{path}"
    _, patient_id, study_id, img_name = path.split("/")
    suffix = img_name.split(".")[-1]
    destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
    if not os.path.exists(destination):
        image = Image.open(source)
        image = resize(image, ratio=0.25)
        image.save(destination)
        diseased_count += 1

print(f"✅ Processed {diseased_count:,} diseased test images (resized)")


🔄 Processing test data - Diseased images (resizing to 25%)...


Processing diseased test images: 100%|█████████████████████████████████████████████████████████| 361/361 [00:17<00:00, 21.09it/s]

✅ Processed 352 diseased test images (resized)


In [12]:
# Process test data - Healthy images (with resizing)
print("\n🔄 Processing test data - Healthy images (resizing to 25%)...")
print("=" * 66)

target_dir = "test/healthy"
create_dir(target_dir)

healthy_count = 0
for path in tqdm(test_paths[healthy_idxs], desc="Processing healthy test images"):
    source = f"../CheXpert-v1.0-small/{path}"
    _, patient_id, study_id, img_name = path.split("/")
    suffix = img_name.split(".")[-1]
    destination = f"{target_dir}/{patient_id}_{study_id}.{suffix}"
    
    if not os.path.exists(destination):
        image = Image.open(source)
        image = resize(image, ratio=0.25)
        image.save(destination)
        healthy_count += 1

print(f"✅ Processed {healthy_count:,} healthy test images (resized)")


🔄 Processing test data - Healthy images (resizing to 25%)...


Processing healthy test images: 100%|██████████████████████████████████████████████████████████| 157/157 [00:06<00:00, 25.21it/s]

✅ Processed 148 healthy test images (resized)


## Final Dataset Summary

Let's verify the final dataset structure and get a comprehensive summary:

In [13]:
# Final Dataset Verification and Summary
print("📋 Final Dataset Summary")
print("=" * 60)

def count_files(directory):
    """Count files in a directory"""
    return len(list(Path(directory).glob("*"))) if Path(directory).exists() else 0

# Define dataset paths
dataset_paths = {
    "Training - Healthy": "train/healthy",
    "Training - Diseased": "train/diseased", 
    "Validation - Healthy": "val/healthy",
    "Validation - Diseased": "val/diseased",
    "Testing - Healthy": "test/healthy",
    "Testing - Diseased": "test/diseased"
}

# Count files and display summary
total_train = 0
total_val = 0
total_test = 0

print("📁 Dataset Structure:")
for label, path in dataset_paths.items():
    count = count_files(path)
    print(f"   {label:<22}: {count:>6} images")
    
    if "Training" in label:
        total_train += count
    elif "Validation" in label:
        total_val += count
    else:
        total_test += count

print("-" * 60)
print(f"{'Total Training':<22}: {total_train:>6} images")
print(f"{'Total Validation':<22}: {total_val:>6} images")
print(f"{'Total Testing':<22}: {total_test:>6} images")
print(f"{'Total Dataset':<22}: {total_train + total_val + total_test:>6} images")

# Calculate class distribution
train_healthy = count_files("train/healthy")
train_diseased = count_files("train/diseased")
val_healthy = count_files("val/healthy")
val_diseased = count_files("val/diseased")
test_healthy = count_files("test/healthy")
test_diseased = count_files("test/diseased")

print("\n📊 Class Distribution:")
print("   Training Set:")
if total_train > 0:
    print(f"      Healthy  : {train_healthy:>6} ({train_healthy/total_train*100:.1f}%)")
    print(f"      Diseased : {train_diseased:>6} ({train_diseased/total_train*100:.1f}%)")
print("   Validation Set:")
if total_val > 0:
    print(f"      Healthy  : {val_healthy:>6} ({val_healthy/total_val*100:.1f}%)")
    print(f"      Diseased : {val_diseased:>6} ({val_diseased/total_val*100:.1f}%)")
print("   Testing Set:")
if total_test > 0:
    print(f"      Healthy  : {test_healthy:>6} ({test_healthy/total_test*100:.1f}%)")
    print(f"      Diseased : {test_diseased:>6} ({test_diseased/total_test*100:.1f}%)")

print("\n✅ CheXpert dataset preprocessing completed successfully!")


📋 Final Dataset Summary
📁 Dataset Structure:
   Training - Healthy    :  11924 images
   Training - Diseased   : 106584 images
   Validation - Healthy  :     30 images
   Validation - Diseased :    170 images
   Testing - Healthy     :    148 images
   Testing - Diseased    :    352 images
------------------------------------------------------------
Total Training        : 118508 images
Total Validation      :    200 images
Total Testing         :    500 images
Total Dataset         : 119208 images

📊 Class Distribution:
   Training Set:
      Healthy  :  11924 (10.1%)
      Diseased : 106584 (89.9%)
   Validation Set:
      Healthy  :     30 (15.0%)
      Diseased :    170 (85.0%)
   Testing Set:
      Healthy  :    148 (29.6%)
      Diseased :    352 (70.4%)

✅ CheXpert dataset preprocessing completed successfully!
